# Optimizar punto de corte, usando semillero para crear multiples learboard Publicos/Privados

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
dataset_file = 'Competencia_01_tagg.csv' # N_cliente, clase_ternaria para junio
dataset_file_shap = 'ensamble_shpa_200_semillas.csv' #N_cliente, Probabiliadd
dataset_file_gv = 'ensamble_gustavo_200_semillas.csv' #N_cliente, Probabiliadd
dataset_file_ensamble='ensamble_shap_gustavo.csv'


clientes_baja_junio= pd.read_csv(dataset_path + dataset_file)
data_shap = pd.read_csv(dataset_path + dataset_file_shap)
data_gv = pd.read_csv(dataset_path + dataset_file_gv)
ensamble= pd.read_csv(dataset_path + dataset_file_gv)

In [28]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    bajas: Columnas: "numero_de_cliente, clase_ternaria"
    prediccion: "numero_de_cliente, Probabilidad"
    corte: cantidad de estimulos
    '''
    # 1. Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=123
    )

    # 2. Obtener los estímulos según el modelo
    estimulos = prediccion.iloc[:corte] 

    # 3. Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # 4. Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    print(f"Ganancia público normalizada: {ganancia_publico}") , print(f"Ganancia privado normalizada: {ganancia_privado}")

In [30]:
calculoGanancia(clientes_baja_junio,data_shap,11000)

Ganancia público normalizada: 87406666.66666667
Ganancia privado normalizada: 96540000.0


## Tengo el problema que segun la semilla con la que dividi el publico privado me puede dar distinto.. hago semillero

In [32]:
def calculoGanancia(bajas, prediccion, corte, random_state):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia normalizada para el conjunto público.
    ganancia_privado: Ganancia normalizada para el conjunto privado.
    '''
    # 1. Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=random_state
    )

    # 2. Obtener los estímulos según el modelo
    estimulos = prediccion.iloc[:corte] 

    # 3. Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # 4. Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Calculamos la ganancia sin normalizar
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

def calculoGananciaPromedio(bajas, prediccion, corte, n_seeds=200):
    '''
    Calcula la ganancia promedio para múltiples semillas aleatorias.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    n_seeds: int, número de semillas aleatorias (por defecto 200).
    
    Retorna:
    promedio_ganancia_publico: Ganancia promedio normalizada para el conjunto público.
    promedio_ganancia_privado: Ganancia promedio normalizada para el conjunto privado.
    '''
    # Generar 200 semillas aleatorias 
    seeds = np.random.randint(1000, 100001, size=n_seeds)
    ganancias_publico = []
    ganancias_privado = []

    for seed in seeds:
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, prediccion, corte, random_state=seed)
        ganancias_publico.append(ganancia_publico)
        ganancias_privado.append(ganancia_privado)

    promedio_ganancia_publico = np.mean(ganancias_publico)
    promedio_ganancia_privado = np.mean(ganancias_privado)

    print(f"Ganancia promedio público: {promedio_ganancia_publico}")
    print(f"Ganancia promedio privado: {promedio_ganancia_privado}")

    return promedio_ganancia_publico, promedio_ganancia_privado


In [35]:
calculoGananciaPromedio(clientes_baja_junio,ensamble,11200)

Ganancia promedio público: 92520866.66666669
Ganancia promedio privado: 92748200.0


(np.float64(92520866.66666669), np.float64(92748200.0))